In [5]:
from sympy import var, Lambda, exp, log, sin, cos, tan, sqrt, ln
from Materia_da_P4 import *

# Resolvendo exercicios da aula

# Fluxo de carga

In [6]:
A9P2 = Fluxo_de_carga(3)

In [7]:
y12 = complex(0.012, 0.3)**(-1)
y23 = complex(0.011, 0.8)**(-1)

Y = [[y12, -y12, 0],
     [-y12, y12 + y23, -y23],
     [0, -y23, y23]]